In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

In [3]:
# prepare data
mnist_dataset = input_data.read_data_sets("MNIST_data/", one_hot=True)
train_X, train_Y, test_X, test_Y = mnist_dataset.train.images,\
mnist_dataset.train.labels, mnist_dataset.test.images, mnist_dataset.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
# define hyper params
input_dim = train_X.shape[1]
hidden_dim = 1024
num_classes = 10
lr = 0.01
epochs = 50
batch_size = 64
iters_per_epoch = train_X.shape[0] // batch_size

# define placeholders and variables
X = tf.placeholder(tf.float32, [None, input_dim])
Y = tf.placeholder(tf.float32, [None, num_classes])
w1 = tf.Variable(tf.random_normal([input_dim, hidden_dim], stddev=0.01), name="weights_1")
w2 = tf.Variable(tf.random_normal([hidden_dim, num_classes], stddev=0.01), name="weights_2")
b1 = tf.Variable(tf.ones([hidden_dim]), name="biases_1")
b2 = tf.Variable(tf.ones([num_classes]), name="biases_2")

In [5]:
# define network architecture
h = tf.nn.relu(tf.matmul(X, w1) + b1)
pred = tf.nn.relu(tf.matmul(h, w2) + b2)

In [6]:
# define loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=Y))

In [7]:
# define training operation using Adam optimizer
train_op = tf.train.AdadeltaOptimizer(lr).minimize(loss)

In [9]:
loss_log = []
test_loss_log = []
acc_log = []
test_acc_log = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # training & test loop
    for epoch in range(epochs):
        # shuffle dataset for every epoch
        rand_indexes = np.random.permutation(train_X.shape[0])
        
        for it in range(iters_per_epoch):
            sess.run(train_op, feed_dict={X: train_X[rand_indexes][it*batch_size:(it+1)*batch_size], Y: train_Y[rand_indexes][it*batch_size:(it+1)*batch_size]})
            
        tmp_loss = sess.run(loss, feed_dict={X: train_X, Y: train_Y})
        tmp_test_loss = sess.run(loss, feed_dict={X: test_X, Y: test_Y})
        tmp_acc = np.mean(np.argmax(sess.run(pred, feed_dict={X: train_X, Y: train_Y}), axis=1) == np.argmax(train_Y, axis=1))
        tmp_test_acc = np.mean(np.argmax(sess.run(pred, feed_dict={X: test_X, Y: test_Y}), axis=1) == np.argmax(test_Y, axis=1))
        loss_log.append(tmp_loss)
        test_loss_log.append(tmp_test_loss)
        acc_log.append(tmp_acc)
        test_acc_log.append(tmp_test_acc)

        print("epoch:{} acc:{:.3f} loss:{:.3f} test_acc:{:.3f} test_loss:{:.3f}".format(epoch+1, tmp_acc, tmp_loss, tmp_test_acc, tmp_test_loss))

epoch:1 acc:0.645 loss:1.961 test_acc:0.658 test_loss:1.955


KeyboardInterrupt: 